# Get the Weather Description and Amount of Precipitation for Each City

In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
len(cities)

643

In [6]:
# Import the datetime module from the datetime library.
from datetime import datetime

In [7]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=4207db3a377792eb24c750378dd88d03


In [9]:
# Create an empty list to hold the weather data.
city_data = []

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    city_url = url + "&q=" + city

  
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Parse out the needed data, adding dictionary index since it is nested.
        city_weather_description = city_weather["weather"][0]["description"]
        
        # Get the amount of rainfall in inches for the last 3 hours
        try:
            city_rain_inches = city_weather["rain"]["3h"] / 25.4
        
        except KeyError:
            city_rain_inches = 0
            
        # Get the amount of snowfall in inches for the last 3 hours
        try:
            city_snow_inches = city_weather["snow"]["3h"] / 25.4
        
        except KeyError:
            city_snow_inches = 0
                
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Convert rainfall and snowfall to inches
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description,
                          "Rain inches (last 3 hrs)": city_rain_inches,
                          "Snow inches (last 3 hrs)": city_snow_inches})

    # If an error is experienced, skip the city.
    except:
        pass

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,Phan Thiet,VN,2020-04-02 20:50:31,10.93,108.10,74.37,81,26,9.78,scattered clouds,0.000000,0.0
1,Albany,US,2020-04-02 20:52:28,42.60,-73.97,55.40,37,90,19.46,overcast clouds,0.000000,0.0
2,Saskylakh,RU,2020-04-02 20:52:29,71.92,114.08,-11.33,86,12,10.42,few clouds,0.000000,0.0
3,Montelibano,CO,2020-04-02 20:52:29,7.98,-75.42,91.40,50,88,9.37,overcast clouds,0.000000,0.0
4,Vaini,TO,2020-04-02 20:52:29,-21.20,-175.20,75.42,88,100,3.44,moderate rain,0.155906,0.0
5,Grand-Bourg,GP,2020-04-02 20:52:29,15.88,-61.31,84.20,74,75,9.17,broken clouds,0.000000,0.0
6,Rikitea,PF,2020-04-02 20:52:29,-23.12,-134.97,76.96,74,60,6.71,light rain,0.046850,0.0
7,Isangel,VU,2020-04-02 20:52:29,-19.55,169.27,78.08,86,100,12.88,moderate rain,0.200000,0.0
8,Saint-Philippe,RE,2020-04-02 20:52:30,-21.36,55.77,73.40,88,75,5.82,broken clouds,0.000000,0.0
9,Sioux Lookout,CA,2020-04-02 20:52:30,50.10,-91.92,48.20,49,90,10.29,overcast clouds,0.000000,0.0


In [11]:
# Create the output file (CSV).
output_challenge_data_file = "weather_data/WeatherPy_challenge.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_challenge_data_file)

In [20]:
# Create DataFrame to determine how many cities have recorded rainfall or snowfall in the last 3 hours.
cities_rain_snow_df = city_data_df.loc[(city_data_df["Rain inches (last 3 hrs)"] > 0) |
                                       (city_data_df["Snow inches (last 3 hrs)"] > 0)]

# Get the number of cies that have recorded rainfall or snowfall in the last 3 hours.
len(cities_rain_snow_df)

97